In [1]:
import json
from pathlib import Path

# Data 
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from scipy.stats import gaussian_kde
import pyproj

# Visualization and mapping
import matplotlib.pyplot as plt
from matplotlib import cm
import folium
from folium import Map, LayerControl, Marker
from folium.plugins import HeatMap, MarkerCluster
from branca.colormap import linear
import folium
from folium import FeatureGroup, LayerControl, GeoJson, GeoJsonTooltip
import geopandas as gpd
import folium
from folium import CircleMarker
import branca.colormap as cm

# Geospatial utilities
from geopandas.tools import sjoin_nearest

# Importing Data

### File Paths

In [2]:
# Total Score
total_score_path = Path("../../data/maps/total_location_score/total_score_metro_atl.geojson")

# Community Transportation Options
transportation_score_path = Path("../../data/maps/community_transportation_options/transportation_options_score_metro_atl.geojson")

# Desirable/Undesirable Activities
desirable_undesirable_score_path = Path("../../data/maps/desirable_undesirable_activities/desirable_undesirable_score_metro_atl.geojson")
desirable_places_path = Path("../../data/maps/desirable_undesirable_activities/desirable_places_metro_atl.geojson")
undesirable_places_path = Path("../../data/maps/desirable_undesirable_activities/undesirable_places_metro_atl.geojson")
food_deserts_path = Path("../../data/maps/desirable_undesirable_activities/food_deserts_metro_atl.geojson")

# Housing Need Characteristics
housing_need_score_path = Path("../../data/maps/housing_need_characteristics/housing_need_score_metro_atl.geojson")
housing_need_indicators_path = Path("../../data/maps/housing_need_characteristics/housing_need_indicators_metro_atl.geojson")

# Quality Education
education_score_path = Path("../../data/maps/quality_education_areas/education_score_metro_atl.geojson")

# Stable Communities
stable_communities_score_path = Path("../../data/maps/stable_communities/stable_communities_score_metro_atl.geojson")
environmental_index_path = Path("../../data/maps/stable_communities/environmental_health_index_metro_atl.geojson")
jobs_index_path = Path("../../data/maps/stable_communities/jobs_proximity_index_metro_atl.geojson")
poverty_index_path = Path("../../data/maps/stable_communities/above_poverty_level_metro_atl.geojson")
income_index_path = Path("../../data/maps/stable_communities/median_income_metro_atl.geojson")
transit_index_path = Path("../../data/maps/stable_communities/transit_access_index_metro_atl.geojson")

# Applicants
applicants_path = Path("../../data/maps/application_list_2022_2023_2024_metro_atl.geojson")

### Load Data

In [3]:
gdf_total_score = gpd.read_file(total_score_path)

gdf_transportation_score = gpd.read_file(transportation_score_path)

gdf_desirable_undesirable_score = gpd.read_file(desirable_undesirable_score_path)
gdf_desirable_places = gpd.read_file(desirable_places_path)
gdf_undesirable_places = gpd.read_file(undesirable_places_path)
gdf_food_deserts = gpd.read_file(food_deserts_path).to_crs("EPSG:4326")

gdf_housing_need_score = gpd.read_file(housing_need_score_path).to_crs("EPSG:4326")
gdf_housing_need_indicators = gpd.read_file(housing_need_indicators_path).to_crs("EPSG:4326")

gdf_education_score = gpd.read_file(education_score_path)

gdf_stable_communities_score = gpd.read_file(stable_communities_score_path).to_crs("EPSG:4326")
gdf_environmental_index = gpd.read_file(environmental_index_path).to_crs("EPSG:4326")
gdf_jobs_index = gpd.read_file(jobs_index_path).to_crs("EPSG:4326")
gdf_poverty_index = gpd.read_file(poverty_index_path).to_crs("EPSG:4326")
gdf_income_index = gpd.read_file(income_index_path).to_crs("EPSG:4326")
gdf_transit_index = gpd.read_file(transit_index_path).to_crs("EPSG:4326")

gdf_applicants = gpd.read_file(applicants_path).to_crs("EPSG:4326")

# Layer Functions

## Circle Layer - Lat/Lon Level

In [4]:
def add_lat_lon_score_layer(gdf, layer_name, score_column="score", palette=None, radius=3):
    """
    Adds a CircleMarker layer to a Folium map using actual scores for colour mapping.

    Args:
        gdf (GeoDataFrame): Must contain geometry and a numeric score column.
        layer_name (str): Name for the layer.
        score_column (str): Name of the column containing numeric scores.
        palette (list): List of hex colours (e.g., YlGnBu_20).
        radius (int): Circle radius in pixels.

    Returns:
        A tuple: (FeatureGroup layer, colourmap)
    """
    # Filter out rows with missing geometry or score
    valid_gdf = gdf[~gdf.geometry.is_empty & gdf.geometry.notnull()].copy()
    valid_gdf = valid_gdf[valid_gdf[score_column].notnull()]
    
    if valid_gdf.empty:
        return folium.FeatureGroup(name=layer_name), None

    # Create colourmap using the score range and your custom palette
    colourmap = cm.LinearColormap(
        colors=palette,
        vmin=valid_gdf[score_column].min(),
        vmax=valid_gdf[score_column].max(),
        caption=layer_name
    )

    # Create a FeatureGroup to hold the markers
    layer = folium.FeatureGroup(name=layer_name)

    # Add each point as a circle marker
    for _, row in valid_gdf.iterrows():
        lat = row.geometry.y
        lon = row.geometry.x
        score = row[score_column]
        colour = colourmap(score)

        CircleMarker(
            location=[lat, lon],
            radius=radius,
            color=colour,
            weight = 0.1,
            fill=True,
            fill_color=colour,
            fill_opacity=0.4,
            popup=f"{score_column.title()}: {score:.2f}"
        ).add_to(layer)

    return layer, colourmap



## Heatmap Layer - Census Tract Level

In [5]:
def add_tract_score_layer_stable(folium_map, gdf, score_column, layer_name, color_scheme="YlGnBu_09"):
    """
    Adds a choropleth-style layer to a Folium map using polygon scores.

    Args:
        folium_map: folium.Map object
        gdf: GeoDataFrame with polygon geometry and a score column
        score_column: name of the column to color by
        layer_name: name of the layer shown in the layer control
        color_scheme: color palette name from branca.linear (default: YlGnBu_09)
    """
    # Ensure CRS is EPSG:4326 for folium
    gdf = gdf.to_crs("EPSG:4326")
    gdf[score_column] = pd.to_numeric(gdf[score_column], errors="coerce")
    print(gdf[score_column].head())
    # Get color scale
    vals = gdf[score_column].dropna()
    if vals.empty:
        print(f"No valid numeric values for '{score_column}' — skipping layer: {layer_name}")
        return

    # Proceed only if there are numeric values left
    if len(vals) == 0:
        print(f"No valid numeric values for '{score_column}' — skipping layer: {layer_name}")
        return
    print("Unique geometry types →", gdf.geom_type.unique())

    cmap = getattr(linear, color_scheme).scale(vals.min(), vals.max())
    cmap.caption = layer_name

    # Add GeoJSON layer
    folium.GeoJson(
        gdf,
        name=layer_name,
        style_function=lambda feature: {
            "fillColor": cmap(feature["properties"][score_column]) if feature["properties"][score_column] is not None else "#d3d3d3",
            "color": "gray",
            "weight": 1,
            "fillOpacity": 0.8
        },
        tooltip=folium.features.GeoJsonTooltip(
            fields=["GEOID", score_column],
            aliases=["Tract", layer_name],
            localize=True
        ),
        options={"name": layer_name}
    ).add_to(folium_map)

    # Add legend
    cmap.add_to(folium_map)

In [6]:
def add_tract_score_layer(folium_map, gdf, score_column, layer_name, color_scheme="YlGnBu_09"):
    """
    Adds a choropleth-style layer to a Folium map using polygon scores.

    Args:
        folium_map: folium.Map object
        gdf: GeoDataFrame with polygon geometry and a score column
        score_column: name of the column to color by
        layer_name: name of the layer shown in the layer control
        color_scheme: color palette name from branca.linear (default: YlGnBu_09)
    """
    # Ensure CRS is EPSG:4326 for folium
    gdf = gdf.to_crs("EPSG:4326")
    gdf[score_column] = pd.to_numeric(gdf[score_column], errors="coerce")
    print(gdf[score_column].head())
    # Get color scale
    vals = gdf[score_column].dropna()
    if vals.empty:
        print(f"No valid numeric values for '{score_column}' — skipping layer: {layer_name}")
        return

    # Proceed only if there are numeric values left
    if len(vals) == 0:
        print(f"No valid numeric values for '{score_column}' — skipping layer: {layer_name}")
        return
    print("Unique geometry types →", gdf.geom_type.unique())

    cmap = getattr(linear, color_scheme).scale(vals.min(), vals.max())
    cmap.caption = layer_name

    # Add GeoJSON layer
    folium.GeoJson(
        gdf,
        name=layer_name,
        style_function=lambda feature: {
            "fillColor": cmap(feature["properties"][score_column]) if feature["properties"][score_column] is not None else "#d3d3d3",
            "color": "gray",
            "weight": 0.3,
            "fillOpacity": 0.8
        },
        tooltip=folium.features.GeoJsonTooltip(
            fields=["GEOID", score_column],
            aliases=["Tract", layer_name],
            localize=True
        ), 
    ).add_to(folium_map)

    # Add legend
    cmap.add_to(folium_map)

## Marker Layer

In [32]:
def add_coloured_markers_to_map(
    folium_map,
    gdf,
    lat_col="lat",
    lon_col="lon",
    colour_by=None,
    popup_fields=None,
    layer_name="Markers",
    clustered=False,
    categorical_colours=None,
):
    """
    Add color-coded markers to a folium map for categorical values.

    Args:
        folium_map: folium.Map object
        gdf: GeoDataFrame or DataFrame
        lat_col: column for latitude
        lon_col: column for longitude
        colour_by: column with categorical values to color by
        popup_fields: list of columns to include in popup
        layer_name: name of the feature group
        clustered: whether to use MarkerCluster
        categorical_colours: dict mapping category to color (optional)
    """
    feature_group = folium.FeatureGroup(name=layer_name)
    if clustered:
        marker_layer = MarkerCluster()
    else:
        marker_layer = folium.FeatureGroup(name=f"{layer_name} Markers")

    if colour_by and categorical_colours is None:
        # Auto-assign colors if not provided
        unique_vals = gdf[colour_by].dropna().unique()
        color_palette = ["green", "red", "blue", "orange", "purple", "gray"]
        categorical_colours = {
            val: color_palette[i % len(color_palette)]
            for i, val in enumerate(sorted(unique_vals))
        }

    for _, row in gdf.iterrows():
        lat = row[lat_col]
        lon = row[lon_col]

        if pd.isnull(lat) or pd.isnull(lon):
            continue  

        colour = "red"
        if colour_by:
            value = row.get(colour_by)
            colour = categorical_colours.get(value, "red")

        # Extract tooltip content
        year = row.get("year", "")
        dev_name = row.get("development_name", "")
        owner_name = row.get("ownership_entity_name", "")
        status = row.get("status", "Unknown")
        dca_score = row.get("dca_score", "")

        tooltip_text = (
            f"<b>{dev_name}</b><br>"
            f"Owner: {owner_name}<br>"
            f"Year: {year}<br>"
            f"Status: {status}<br>"
            f"DCA Score: {dca_score}"
        )

        folium.CircleMarker(
            location=(lat, lon),
            radius=2,
            color=colour,
            fill=True,
            fill_color=colour,
            fill_opacity=1,
            weight=0,
            tooltip=folium.Tooltip(tooltip_text, sticky=True)  # ✅ This enables the hover effect
        ).add_to(marker_layer)

    marker_layer.add_to(feature_group)
    feature_group.add_to(folium_map)


# Build Maps

## Location-Based Score Maps

In [33]:
center_lat = gdf_total_score["lat"].mean()
center_lon = gdf_total_score["lon"].mean()

m_location_score = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=9,
        tiles="CartoDB positron",   
        attr="CartoDB positron"    
)

YlGnBu_20 = ['#ffffd9', '#f7fcbf', '#eaf8b8', '#d5efb1', '#b7e3ac',
 '#94d6a8', '#6ec7a7', '#4db7a6', '#30a4a6', '#1991a6',
 '#107fa8', '#126ea8', '#165da8', '#194ea7', '#1c3fa5',
 '#1e30a1', '#1d239a', '#1a1790', '#150e83', '#0d0575']

YlGnBu_5 = ['#ffffd9', '#d9efb2', '#94d6a8', '#2ba5b4', '#0d0575']

status_colours = {
    "Select": "#7CFC00",
    "Non-select": "red"
}

In [34]:
circle_total, colourbar_total = add_lat_lon_score_layer(
    gdf_total_score,
    layer_name="Total Score",
    score_column="score",
    palette=YlGnBu_20,
    radius=4
)

circle_total.add_to(m_location_score)
if colourbar_total:
    colourbar_total.add_to(m_location_score)

In [35]:
circle_du, colourbar_du = add_lat_lon_score_layer(
    gdf_desirable_undesirable_score,
    layer_name="Desirable/Undesirable Activities Score",
    score_column="score",
    palette=YlGnBu_20,
    radius=4
)

circle_du.add_to(m_location_score)
if colourbar_du:
    colourbar_du.add_to(m_location_score)

In [36]:
circle_transport, colourbar_transport = add_lat_lon_score_layer(
    gdf_transportation_score,
    layer_name="Community Transportation Score",
    score_column="score",
    palette=YlGnBu_5,
    radius=4
)

circle_transport.add_to(m_location_score)
if colourbar_transport:
    colourbar_transport.add_to(m_location_score)

In [37]:
add_tract_score_layer_stable(
    m_location_score,
    gdf_stable_communities_score,          
    score_column="score",
    layer_name="Stable Communities Score", 
)

0     6.0
1    10.0
2    10.0
3     8.0
4     8.0
Name: score, dtype: float64
Unique geometry types → ['Polygon' 'MultiPolygon']


In [38]:
add_tract_score_layer(
    m_location_score,
    gdf_education_score,          
    score_column="score",
    layer_name="Quality Education Score", 
)

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: score, dtype: float64
Unique geometry types → ['Polygon' 'MultiPolygon']


In [39]:
add_coloured_markers_to_map(
    folium_map=m_location_score,
    gdf=gdf_applicants,
    lat_col="lat",
    lon_col="lon",
    colour_by="status",  
    popup_fields=None,
    layer_name="Applicant Locations",
    categorical_colours=status_colours,
)
folium.LayerControl(collapsed=False, position="topright").add_to(m_location_score)
m_location_score.save("../../maps/location_score_map.html")

# Stable Communities and Indicators Maps

In [15]:
center_lat = gdf_total_score["lat"].mean()
center_lon = gdf_total_score["lon"].mean()

m_stable_communities = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=9,
        tiles="CartoDB dark_matter",   
        attr="CartoDB dark_matter"    
)

status_colours = {
    "Select": "#7CFC00",
    "Non-select": "red"
}


In [16]:
add_tract_score_layer_stable(
    m_stable_communities,
    gdf_stable_communities_score,          
    score_column="score",
    layer_name="Stable Communities Score", 
)

0     6.0
1    10.0
2    10.0
3     8.0
4     8.0
Name: score, dtype: float64
Unique geometry types → ['Polygon' 'MultiPolygon']


In [17]:
add_tract_score_layer(
    m_stable_communities,
    gdf_environmental_index,          
    score_column="Environmental Health Index",
    layer_name="Environmental Health Index", 
)

0    0.419293
1    0.264565
2    0.199618
3    0.162369
4    0.498567
Name: Environmental Health Index, dtype: float64
Unique geometry types → ['Polygon' 'MultiPolygon']


In [18]:
add_tract_score_layer_stable(
    m_stable_communities,
    gdf_poverty_index,          
    score_column="Percent of Population Above the Poverty Level",
    layer_name="% of Population Above the Poverty Level", 
)

0    93.7
1    94.3
2    97.1
3    97.2
4    81.6
Name: Percent of Population Above the Poverty Level, dtype: float64
Unique geometry types → ['Polygon' 'MultiPolygon']


In [19]:
add_tract_score_layer(
    m_stable_communities,
    gdf_jobs_index,          
    score_column="Jobs Proximity Index",
    layer_name="Jobs Proximity Index", 
)

0    0.539365
1    0.674729
2    0.698227
3    0.722188
4    0.627898
Name: Jobs Proximity Index, dtype: float64
Unique geometry types → ['Polygon' 'MultiPolygon']


In [20]:
add_tract_score_layer(
    m_stable_communities,
    gdf_transit_index,          
    score_column="Transit Access Index",
    layer_name="Transit Access Index", 
)

0    0.469359
1    0.644534
2    0.686631
3    0.535469
4    0.618557
Name: Transit Access Index, dtype: float64
Unique geometry types → ['Polygon' 'MultiPolygon']


In [21]:
add_tract_score_layer(
    m_stable_communities,
    gdf_income_index,          
    score_column="Median Income",
    layer_name="Median Income"
)

0     73406.0
1    120694.0
2    110066.0
3    110799.0
4     47383.0
Name: Median Income, dtype: float64
Unique geometry types → ['Polygon' 'MultiPolygon']


In [22]:
add_coloured_markers_to_map(
    folium_map=m_stable_communities,
    gdf=gdf_applicants,
    lat_col="lat",
    lon_col="lon",
    colour_by="status",  
    popup_fields=None,
    layer_name="Applicant Locations",
    categorical_colours=status_colours,
)
folium.LayerControl(collapsed=False, position="topright").add_to(m_stable_communities)
m_stable_communities.save("../../maps/stable_communities_map.html")

#### Housing Needs Characteristics Indicator

In [23]:
center_lat = gdf_total_score["lat"].mean()
center_lon = gdf_total_score["lon"].mean()

m_housing_needs = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=9,
        tiles="CartoDB positron",   
        attr="CartoDB positron"    
)


status_colours = {
    "Select": "#7CFC00",
    "Non-select": "red"
}

In [24]:
add_tract_score_layer(
    m_housing_needs,
    gdf_housing_need_indicators,          
    score_column="% of rental units occupied by 80% AMI and below with Severe Housing Problems",
    layer_name="% of rental units occupied by 80% AMI and below with Severe Housing Problems"
)

0    46.875000
1    54.285714
2    56.410256
3     0.000000
4    42.471910
Name: % of rental units occupied by 80% AMI and below with Severe Housing Problems, dtype: float64
Unique geometry types → ['Polygon' 'MultiPolygon']


In [25]:
add_coloured_markers_to_map(
    folium_map=m_housing_needs,
    gdf=gdf_applicants,
    lat_col="lat",
    lon_col="lon",
    colour_by="status",  
    popup_fields=None,
    layer_name="Applicant Locations",
    categorical_colours=status_colours,
)
folium.LayerControl(collapsed=False, position="topright").add_to(m_housing_needs)
m_housing_needs.save("../../maps/housing_needs_map.html")

# Kernel Density Estimation
### For lon/lat data

In [26]:
# def compute_kde_heatmap_data(gdf, value_column, grid_size=500, bandwidth_scale=0.1):
#     """
#     Projects gdf, applies KDE on the specified value_column,
#     and returns [lat, lon, density] points for folium HeatMap.

#     Args:
#         gdf: GeoDataFrame with Point geometry in EPSG:4326
#         value_column: string name of the score column to smooth
#         grid_size: resolution of KDE grid (default: 500 x 500)
#         bandwidth_scale: multiplier for KDE bandwidth (default: 0.1)

#     Returns:
#         List of [lat, lon, kde_value] points
#     """
    
#     # Project to meters
#     gdf_proj = gdf.to_crs("EPSG:3857")
#     gdf_proj["x"] = gdf_proj.geometry.x
#     gdf_proj["y"] = gdf_proj.geometry.y

#     # Set up KDE inputs
#     xy = np.vstack([gdf_proj["x"], gdf_proj["y"]])
#     weights = gdf_proj[value_column]

#     kde = gaussian_kde(xy, weights=weights)
#     kde.set_bandwidth(kde.factor * bandwidth_scale)

#     # Create evaluation grid
#     x_min, y_min, x_max, y_max = gdf_proj.total_bounds
#     xx, yy = np.mgrid[x_min:x_max:grid_size*1j, y_min:y_max:grid_size*1j]
#     grid_coords = np.vstack([xx.ravel(), yy.ravel()])
#     zz = kde(grid_coords).reshape(xx.shape)

#     # Normalize
#     zz_scaled = zz / zz.max()

#     # Convert back to lat/lon
#     transformer = pyproj.Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)
#     lon_grid, lat_grid = transformer.transform(xx, yy)

#     # Create [lat, lon, value] output
#     heatmap_data = [
#         [lat_grid[i, j], lon_grid[i, j], zz_scaled[i, j]]
#         for i in range(xx.shape[0])
#         for j in range(xx.shape[1])
#         if zz_scaled[i, j] > 0
#     ]

#     return heatmap_data

#### Base Map

In [27]:
# m_desirable_undesirable = folium.Map(location=[32.5, -83.5], zoom_start=7, tiles="CartoDB positron")

#### Add KDE Heatmap Layers

In [28]:
# def add_kde_heatmap_layer(folium_map, heatmap_data, layer_name, radius=15, blur=25):
#     """
#     Adds a KDE-based HeatMap layer to a Folium map.

#     Args:
#         folium_map: The folium.Map object to add to
#         heatmap_data: List of [lat, lon, value] from compute_kde_heatmap_data
#         layer_name: Name of the heatmap layer
#         radius: HeatMap radius in pixels
#         blur: HeatMap blur in pixels
#     """
#     HeatMap(
#         heatmap_data,
#         name=layer_name,
#         radius=radius,
#         blur=blur,
#         min_opacity=0.3,
#     ).add_to(folium_map)

In [29]:
# Run for multiple metrics
# datasets = {
#     "Desirable/Undesirable Activities": {
#         "gdf": gdf_desirable_filtered,
#         "metrics": {
#             "score": "Desirable/Undesirable Activities Score"
#         }
#     },
# }

# for dataset_name, config in datasets.items():
#     gdf = config["gdf"]
#     for col, layer_name in config["metrics"].items():
#         print(f"Adding {layer_name} from {dataset_name}...")
#         kde_data = compute_kde_heatmap_data(gdf, value_column=col, bandwidth_scale=0.05)
#         add_kde_heatmap_layer(m_desirable_undesirable, kde_data, layer_name=layer_name)

# folium.LayerControl(collapsed=False).add_to(m_desirable_undesirable)

# m_desirable_undesirable.save("../../maps/heatmap_desirable_undesriable_score.html")